# <B> Setup for Nvidai NeMo with SageMaker </B>
* Container: codna_python3

## AutoReload

In [ ]:
%load_ext autoreload
%autoreload 2

## 0. Install packages

In [ ]:
install_needed = True  # should only be True once
# install_needed = False

In [ ]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

In [ ]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U smdebug sagemaker-experiments
    !{sys.executable} -m pip install -U sagemaker
    !{sys.executable} -m pip install -U datasets transformers
    !{sys.executable} -m pip install -U wget omegaconf text-unidecode sox
    
    ## Install NeMo
    !sudo yum install sox -y
    !sudo yum install libsndfile
    !pip install --upgrade --force-reinstall llvmlite
    BRANCH = 'main'
    !{sys.executable} -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]
    
    IPython.Application.instance().kernel.do_shutdown(True)

## 1. Set roles

In [ ]:
from sagemaker import get_execution_role

In [ ]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

## 1.1 Attach IAM polich to sagemaker execution role (<b>with console</b>)
> **EC2ContainerRegistry**: "arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess" <BR>
> **S3**: "arn:aws:iam::aws:policy/AmazonS3FullAccess"

## 2. Set default parameters

In [ ]:
import boto3
import sagemaker

### Bucket / Prefix 설정

In [ ]:
strRegionName = boto3.Session().region_name
strAccountId = boto3.client("sts").get_caller_identity().get("Account")
bucket_name = ' ' # <-- 사용할 bucket 명을 추가해 주세요. ex) sagemaker-us-east-1-123456789123, sm-nemo-bucket
prefix = ' ' ## <-- 작업할 prefix 명을 추가해 주세요. ex) nemo-test, nemo-asr

## 3. Create training custom docker image

* docker build

In [ ]:
!pygmentize custom-docker/Dockerfile

* Base 이미지의 region, account-id 확인 후 아래 파라미터 입력

### 1) AWS CLI 를 이용한 방식

In [ ]:
%%bash
strRepositoryName="nemo-test-training"  ## <-- 원하는 docker repostory 이름을 추가
strDockerDir="./custom-docker/"
strTag="latest"

cd ${strDockerDir}
echo $(pwd)
container_name=${strRepositoryName}

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
# region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${container_name}:${strTag}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${container_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${container_name}" > /dev/null
fi

# # Get the login command from ECR and execute it directly
# $(aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin "763104351884.dkr.ecr.us-west-2.amazonaws.com")

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker build -f Dockerfile -t ${fullname} .
# docker tag ${container_name} ${fullname}

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)
docker push ${fullname}
echo ${fullname}

In [ ]:
strEcrRepositoryUri = ' '#' '  ## <-- 생성된 ECR의 URI를 넣어주세요.. ex) 123456789123.dkr.ecr.us-west-2.amazonaws.com/nemo-test:latest

### 2) AWS BOTO3 SDK를 이용한 방식

In [ ]:
from utils.ecr import ecr_handler
ecr = ecr_handler()

In [ ]:
strRepositoryName="nemo-test-training"  ## <-- 원하는 docker repostory 이름을 추가
strRepositoryName = strRepositoryName.lower()
strDockerDir = "./custom-docker/"
strDockerFile = "Dockerfile"
strTag = "latest"

In [ ]:
ecr.build_docker(strDockerDir, strDockerFile, strRepositoryName, strRegionName="us-west-2", strAccountId="763104351884")

* Push the image to ECR

In [ ]:
strEcrRepositoryUri = ecr.register_image_to_ecr(strRegionName, strAccountId, strRepositoryName, strTag)

* Save image-uri to parameter store

## 4. Create inference custom docker image

* docker build

In [ ]:
!pygmentize custom-docker/Dockerfile.inf

### 1) AWS CLI 를 이용한 방식

In [ ]:
%%bash
strRepositoryName="nemo-test-inference"
strDockerDir="./custom-docker/"
strTag="latest"

cd ${strDockerDir}
echo $(pwd)
container_name=${strRepositoryName}

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
# region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${container_name}:${strTag}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${container_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${container_name}" > /dev/null
fi

# # Get the login command from ECR and execute it directly
# $(aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin "763104351884.dkr.ecr.us-west-2.amazonaws.com")

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker build -f Dockerfile.inf -t ${fullname} .
# docker tag ${container_name} ${fullname}

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)
docker push ${fullname}
echo ${fullname}

In [ ]:
strInfEcrRepositoryUri = ' '  ## <-- 생성된 ECR의 URI를 넣어주세요.. ex) 123456789123.dkr.ecr.us-west-2.amazonaws.com/nemo-test:latest


### 2) AWS BOTO3 SDK를 이용한 방식

In [ ]:
from utils.ecr import ecr_handler
ecr = ecr_handler()

In [ ]:
strInfRepositoryName="nemo-test-inference"  ## <-- 원하는 docker repostory 이름을 추가
strInfRepositoryName = strInfRepositoryName.lower()
strDockerFile = "Dockerfile.inf"
strDockerDir = "./custom-docker/"
strTag = "latest"

In [ ]:
ecr.build_docker(strDockerDir, strDockerFile, strInfRepositoryName, strRegionName="us-west-2", strAccountId="763104351884")

In [ ]:
strInfEcrRepositoryUri = ecr.register_image_to_ecr(strRegionName, strAccountId, strInfRepositoryName, strTag)

## 5. Download & Upload dataset

In [ ]:
import os
import wget

In [ ]:
data_dir = "./data"

In [ ]:
print("******")
os.makedirs(data_dir, exist_ok=True)
if not os.path.exists(data_dir + '/an4_sphere.tar.gz'):
    an4_url = 'https://dldata-public.s3.us-east-2.amazonaws.com/an4_sphere.tar.gz'
    an4_path = wget.download(an4_url, data_dir)
    print(f"Dataset downloaded at: {an4_path}")
else:
    print("Tarfile already exists.")
    an4_path = data_dir + '/an4_sphere.tar.gz'

In [ ]:
strRepositoryName="nemo-test-inference"  ## <-- 원하는 docker repostory 이름을 추가
strRepositoryName = strRepositoryName.lower()
strDockerDir = "./custom-docker/"
strTag = "latest"

* updoad data to s3

### 1) AWS CLI 를 이용한 방식

In [ ]:
!aws s3 sync $data_dir s3://$bucket_name/$prefix/data --quiet

### 2) AWS BOTO3 SDK를 이용한 방식

In [ ]:
from utils.s3 import s3_handler

In [ ]:
s3 = s3_handler()

In [ ]:
source_dir, target_bucket, target_dir = data_dir, pm.get_params(key=prefix+"-BUCKET"), prefix+"/data"
s3.upload_dir(source_dir, target_bucket, target_dir)

## 6. Upload Pretrained model

In [ ]:
pretrained = os.getcwd() + '/pretrained/CTC.nemo'

pretrained_s3uri = os.path.join(
    "s3://{}".format(bucket_name),
    prefix,
    "pretrained",
)

!aws s3 sync pretrained $pretrained_s3uri

## 7. CodeCommit 생성
- Attach IAM polich to sagemaker execution role (<b>with console</b>)
> **CodeCommit**: "arn:aws:iam::aws:policy/AWSCodeCommitFullAccess"<BR>
> **SecretsManager**: "arn:aws:iam::aws:policy/SecretsManagerReadWrite"<BR>

### 5.1 CodeCommit 관련 Credentials 생성 및 Secret Manager에 저장하기
- CodeCommit Credentials

In [ ]:
user_name = '   ' ## ==> IAM에서 사용자 아이디 확인합니다.
codecommit_cred = 'codecommit-cred-'+user_name
codecommit_cred

In [ ]:
iam_client = boto3.client('iam')

In [ ]:
try:
    response = iam_client.list_service_specific_credentials(
        UserName=user_name,
        ServiceName='codecommit.amazonaws.com'
    )
    if len(response['ServiceSpecificCredentials']) > 0:
        response = iam_client.delete_service_specific_credential(
            UserName=user_name,
            ServiceSpecificCredentialId=response['ServiceSpecificCredentials'][-1]['ServiceSpecificCredentialId']
        )
except:
    print("Create new codecommit crendentials")
    pass
finally:
    response = iam_client.create_service_specific_credential(
        UserName=user_name,
        ServiceName='codecommit.amazonaws.com'
    )
    ServiceUserName = response['ServiceSpecificCredential']['ServiceUserName']
    ServicePassword = response['ServiceSpecificCredential']['ServicePassword']
print(f"ServiceUserName : {ServiceUserName} \nServicePassword : {ServicePassword}")

In [ ]:
code_repository_name = 'nemo-code' ## ==> 사용할 code repository 폴더 명을 넣습니다. ex) model_code
local_code_dir = './code' ## ==> 생성한 local의 code repository 폴더 명을 넣습니다. ex) code

In [ ]:
codecommit = boto3.client('codecommit')

try:
    response = codecommit.create_repository(
        repositoryName=code_repository_name,
        repositoryDescription='Data Scientists share their training code using this Repository'
    )
except:
    
    print("Repository already exists")
    response = codecommit.get_repository(
        repositoryName=code_repository_name
    )

In [ ]:
codecommit_repo = response['repositoryMetadata']['cloneUrlHttp']
codecommit_repo

In [ ]:
!rm -rf .git/

In [ ]:
!git init
!git remote add repo_codecommit $codecommit_repo
!git checkout -b main
!git add $local_code_dir
!git commit -m "code-update"
!git push --set-upstream repo_codecommit main

## 6. [Optional] AWS Systems Manager Parameter Store 를 이용한 파라미터 저장/활용
- [AWS Systems Manager Parameter Store](https://docs.aws.amazon.com/systems-manager/latest/userguide/systems-manager-parameter-store.html)
- Attach IAM polich to sagemaker execution role (<b>with console</b>)
> **SSM**: "arn:aws:iam::aws:policy/AmazonSSMFullAccess"<BR>

In [ ]:
from utils.ssm import parameter_store

In [ ]:
pm = parameter_store(strRegionName)

In [ ]:
pm.put_params(key="PREFIX", value=prefix, overwrite=True)
pm.put_params(key="-".join([prefix, "REGION"]), value=strRegionName, overwrite=True)
pm.put_params(key="-".join([prefix, "BUCKET"]), value=bucket_name, overwrite=True)
pm.put_params(key="-".join([prefix, "SAGEMAKER-ROLE-ARN"]), value=get_execution_role(), overwrite=True)
pm.put_params(key="-".join([prefix, "ACCOUNT-ID"]), value=strAccountId, overwrite=True)
pm.put_params(key="-".join([prefix, "IMAGE-URI"]), value=strEcrRepositoryUri, overwrite=True)
pm.put_params(key="-".join([prefix, "INF-IMAGE-URI"]), value=strInfEcrRepositoryUri, overwrite=True)
pm.put_params(key="-".join([prefix, "S3-DATA-PATH"]), value=f"s3://{bucket_name}/{prefix}/data", overwrite=True)
pm.put_params(key="-".join([prefix, "CODE_REPO"]), value=codecommit_repo.replace('https://',''), overwrite=True)  ## https:// 있을 경우 입력 못함
pm.put_params(key="-".join([prefix, "CODECOMMIT-USERNAME"]), value=ServiceUserName, overwrite=True, enc=True)
pm.put_params(key="-".join([prefix, "CODECOMMIT-PWD"]), value=ServicePassword, overwrite=True, enc=True)
pm.put_params(key="-".join([prefix, "PRETRAINED-WEIGHT"]), value=pretrained_s3uri, overwrite=True)
pm.put_params(key="-".join([prefix, "RETRAIN"]), value=False, overwrite=True)

In [ ]:
print (f'PREFIX: {pm.get_params(key="PREFIX")}')
print (f'REGION: {pm.get_params(key="-".join([prefix, "REGION"]))}')
print (f'BUCKET: {pm.get_params(key="-".join([prefix, "BUCKET"]))}')
print (f'SAGEMAKER-ROLE-ARN: {pm.get_params(key="-".join([prefix, "SAGEMAKER-ROLE-ARN"]))}')
print (f'ACCOUNT-ID: {pm.get_params(key="-".join([prefix, "ACCOUNT-ID"]))}')
print (f'IMAGE-URI: {pm.get_params(key="-".join([prefix, "IMAGE-URI"]))}')
print (f'INF-IMAGE-URI: {pm.get_params(key="-".join([prefix, "INF-IMAGE-URI"]))}')
print (f'S3-DATA-PATH: {pm.get_params(key="-".join([prefix, "S3-DATA-PATH"]))}')
print (f'CODE_REPO: {pm.get_params(key="-".join([prefix, "CODE_REPO"]))}')
print (f'CODECOMMIT-USERNAME: {pm.get_params(key="-".join([prefix, "CODECOMMIT-USERNAME"]), enc=False)}')
print (f'CODECOMMIT-PWD: {pm.get_params(key="-".join([prefix, "CODECOMMIT-PWD"]), enc=False)}')
print (f'PRETRAINED-WEIGHT: {pm.get_params(key="-".join([prefix, "PRETRAINED-WEIGHT"]))}')
print (f'RETRAIN: {pm.get_params(key="-".join([prefix, "RETRAIN"]))}')